In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
import keras
from keras import layers
from stacked_mnist import StackedMNISTData, DataMode

In [15]:
input_img = keras.Input(shape=(28, 28, 1))

# Encoder
x = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(input_img)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(2, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(2, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((1, 1), padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
encoded = layers.Dense(4, activation="relu")(x)

# Decoder
decoder_input = layers.Dense(128, activation="relu")(encoded)
x = layers.Dense(7*7*2, activation="relu")(decoder_input)
x = layers.Reshape((7, 7, 2))(x)
x = layers.Conv2DTranspose(4, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(8, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(16, (3, 3), strides=2, activation="relu", padding="same")(x)
decoded = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.save_weights('initial_model.h5')
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 14, 14, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 14, 14, 2)         290       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 7, 7, 2)          0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 7, 7, 2)           38  

In [16]:
generator = StackedMNISTData(mode=DataMode.MONO_BINARY_COMPLETE, default_batch_size=2048)

x_train, y_train = generator.get_full_data_set(training=True)
x_test, y_test = generator.get_full_data_set(training=False)
# "Translate": Only look at "red" channel; only use the last digit. Use one-hot for labels during training
x_train = x_train[:, :, :, [0]]
x_test = x_test[:, :, :, [0]]

In [17]:
autoencoder.fit(x_train, x_train,
                epochs=30,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/30


ValueError: in user code:

    File "/usr/local/anaconda3/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/anaconda3/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/anaconda3/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/anaconda3/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "/usr/local/anaconda3/envs/tensorflow/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/anaconda3/envs/tensorflow/lib/python3.8/site-package